In [ ]:
#This file defines the patient agent, doctor agent and laboratory agent,then agents will interact with each other to simulate a medical diagnosis process.

import openai  # OpenAI API is pretty expensive,try to find something to change it
import time

In [ ]:
#delete it before submitting
openai.api_key = ""

In [ ]:
#for judging if the data is valid
import json

file_path = "MedQA_organized.jsonl"

with open(file_path, "r", encoding="utf-8") as f:
    for idx, line in enumerate(f, start=1):
        line = line.strip()
        if not line:
            continue  # 跳过空行
        try:
            obj = json.loads(line)
            print(f"Line {idx} parsed successfully:", obj)
        except json.JSONDecodeError as e:
            print(f"Line {idx} JSONDecodeError: {e}")

In [ ]:
import json

file_path = 'MedQA_organized.jsonl'
data = []
with open(file_path, 'r', encoding='utf-8') as f:
    for line in f:
        # 去掉换行符
        line = line.strip()
        # 如果这行内容不是空的，就解析成 Python 对
        data.append(json.loads(line))
        print(data)


In [ ]:
print(data[2])

In [47]:
class DataDistributor:
    """
    Structure data from a json object.
    """
    def __init__(self, json_object):
        self.patient_information = json_object['Patient Information']
        self.disease = json_object['Correct_Diagnosis']
        self.physical_examination_findings=json_object['Physical_Examination_Findings']
        self.test_result = json_object['Test_Results']
    
    def get_summary(self):
        """
        Return a summary of the structured data.
        """
        return {
            "Patient Information": self.patient_information,
            "Correct Diagnosis": self.disease,
            "Physical Examination Findings": self.physical_examination_findings,
            "Test Results": self.test_result,
        }
    

In [48]:
a=DataDistributor(data[2])


In [49]:
a.patient_information

{'Gender and Age': '55-year-old man',
 'Patient History': 'The patient has a 6-week history of tingling pain in the sole of his right foot when raised above chest level during exercises, started exercising 2 months ago, right calf cramps when using the incline feature on the treadmill which forces frequent breaks, pain disappears after a few minutes of rest, 8-year history of type 2 diabetes mellitus, smokes two packs of cigarettes daily for 34 years, only medication is metformin.',
 'Chief Complaint': 'Tingling pain in the sole of the right foot when raised above chest level during exercises and right calf cramping on treadmill incline',
 'Secondary Symptoms': 'Pain disappears after resting for a few minutes',
 'Past_Medical_History': '8-year history of type 2 diabetes mellitus',
 'Social_History': 'Smokes two packs of cigarettes daily for 34 years',
 'Review_of_Systems': 'No back pain, normal muscle strength.'}

In [50]:
#逻辑：三个agent，病人、医生和实验室
#实验室是个函数，接受医生的测试请求，返回测试结果。如果没有相关的测试则返回正常。


class Laboratory:
    """
    Return measurement results based on the doctor's request.
    """
    prompt = "你将扮演一个用于医学检查的实验室。"
   
    def __init__(self, data_distributor):
        self.Test_Results = data_distributor.test_result
    def get_result(self, doctor_request):
        """
        Return the result of the test.
        """
        messages = [
        {"role": "system", "content": Laboratory.prompt},
        {"role": "user", "content": f"医生请求检查:{doctor_request}"+
                         f"这是检查结果的列表{self.Test_Results}"+
                         "请直接回复对应的检查结果，不需要添加额外信息"},
            ]
        response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=messages,
            )
            
        return response.choices[0].message['content']
        

In [72]:
def asking_question(model_name,prompt,system_prompt,max_retries=2,time_out=10):
    """
    ask the model and return the answer
    """
    valid_model = ["gpt4o"]
    if model_name not in valid_model:
        raise Exception(f"No model by the name {model_name},all the valid models are {valid_model}")
    
    for _ in range(max_retries):
        try:
            if model_name == "gpt4o":
                print(model_name)
                response = openai.ChatCompletion.create(
                    model="gpt-4o",
                    messages=[
                        {"role": "system", "content": system_prompt},
                        {"role": "user", "content": prompt},
                    ],
                )
                return  response["choices"][0]["message"]["content"]
            
            
        except Exception as e:
            print(e)
            time.sleep(time_out)
            continue

In [87]:
class Patient:
    """
    Return measurement results based on the doctor's request.
    """

    
    def __init__(self, data_distributor, backend="gpt4o"):
        self.info = data_distributor.patient_information
        self.backend = backend
        self.conversation_history = "" #history of conversation
    
    def return_question(self, doctor_question)->str:
        #可以考虑增加历史对话的压缩机制
        """
        Return the answer of doctor's question.
        """
        input_prompt =  "\n下面是你的对话历史: " + self.conversation_history + "\n 下面是医生的回复: "+ doctor_question +"\n"+ "现在请继续你的对话\nPatient: "
        answer = asking_question(self.backend,input_prompt, self.system_prompt())
        self.conversation_history += doctor_question+ "\n\n" + answer + "\n\n"  #继续对话
        return answer
    
    
    def system_prompt(self) -> str:
        #we can add more templates and put them into another file
        core_prompt="""You are role-playing as a patient in a clinic. A doctor is inspecting you to identify your disease by asking questions and performing examinations. Your task is to respond naturally and concisely to the doctor's questions.
            Instructions:
            1. Strictly answer the questions based on the "Patient Information." If the answer to the doctor's question is found in the "Patient Information," respond according to its content. If it is not found in the "Patient Information," respond the doctor with normal information."
            2. Only answer in dialogue form, as if you were speaking directly to the doctor.
            2. Your responses should be brief, realistic, and limited below 4 sentences.
            3. Provide information that a patient might reasonably say in this scenario.
            4. Do not elaborate or provide excessive details unless explicitly asked."""
        
        patient_info_prompt = f"\n\nBelow is all of your information. {self.info}."
        return core_prompt + patient_info_prompt
    
    def reset(self) -> None:
        self.conversation_history = ""


In [88]:
#Now we can test the patient agent
a.patient_information
patient_agent = Patient(a)

In [91]:
patient_agent.return_question("你好，你过去有什么医疗记录吗")

gpt4o


'我已经有8年的2型糖尿病病史。'

In [92]:
patient_agent.conversation_history

'你好，我是你的医生\n\n您好，医生。最近我在做运动时，只要把右脚抬到胸部以上，就会感到脚底刺痛。而且用跑步机的坡度功能时，右小腿会抽筋。休息几分钟后，这些疼痛就会消失。\n\n你好，你还有什么症状吗\n\n没有其他症状了。休息后疼痛会消失，没有背痛，肌肉力量也正常。\n\n你好，你过去有什么医疗记录吗\n\n我已经有8年的2型糖尿病病史。\n\n'

In [ ]:
#some features can be added: 1.choose the backbone LLM 2.choose the prompt template 3.maximum conversation turns
#4. memory storage(maybe can be shared by the patient agent),could add summary mechanism to reduce the memory size

class Doctor:
    """
    Return measurement results based on the doctor's request.
    """
    prompt = "你将扮演一个医生。"
   
    def __init__(self, data):
        self.symptoms = data["symptoms"]
    def get_symptoms(self):
        """
        Return the result of the test.
        """
        messages = [
        {"role": "system", "content": Doctor.prompt},
        {"role": "user", "content": "请描述你的症状"},
            ]
        response = openai.ChatCompletion.create(
                model="gpt-4o",
                messages=messages,
            )
            
        return response.choices[0].message['content']